# Landscape Expansion Index

More details on the wiki - https://github.com/worldbank/GOST_Urban/wiki/Landscape-Expansion-Index

In [3]:
import os
import sys
import importlib
import rasterio
import rasterio.features

import geopandas as gpd
import pandas as pd

sys.path.append("../../../gostrocks/src/")
import GOSTRocks.rasterMisc as rMisc

# Import GOST urban functions
sys.path.append("../../src/GOST_Urban")
import LEI as lei

%load_ext autoreload
%autoreload 2

In [7]:
lei.calculate_LEI?

Signature:
lei.calculate_LEI(
    inputGHSL,
    old_list,
    new_list,
    buffer_dist=300,
    transform='',
)
Docstring:
Generate urban, feature-level LEI; generate area calculations of new and old extents

:param inputGHSL: Path to a geotiff or a rasterio object, or a numpy array containing the categorical
    data used to calculate LEI
:type inputGHSL: rasterio.DatasetReader
:param old_list: Values in built area raster to consider old urban.
:type old_list: list of ints
:param new_list: Values in built area raster to consider new urban
:type new_list: list of ints
:param buffer_dist: distance to buffer new urban areas for comparison to old urban extents, defaults to 300 (m)
:type buffer_dist: int, optional
:param transform: rasterio transformation object. Required if inputGHSL is a numpy array, defaults to ''
:type transform: str, optional
:returns: individual vectors of new built areas with LEI results. Each item is a single new built feature with three columns: 
            1. 

In [ ]:
# Define input variables
input_folder = "/home/wb411133/temp/LOME"
if not os.path.exists(input_folder):
    os.makedirs(input_folder)
input_ghsl = os.path.join(input_folder, "GHSL.tif")

# This section will extract GHSL data from the global file, if you have the GHSL for the AOI extracted
#    define above as input_ghsl
if not os.path.exists(input_ghsl):
    # clip from global GHSL file
    ghsl_vrt = "/home/public/Data/GLOBAL/GHSL/ghsl.vrt"
    aoi = os.path.join(input_folder, "Grand_lome_dissolve.shp")
    in_ghsl = rasterio.open(ghsl_vrt)
    inA = gpd.read_file(aoi)
    if not inA.crs == in_ghsl.crs:
        inA = inA.to_crs(in_ghsl.crs)
    rMisc.clipRaster(in_ghsl, inA, input_ghsl)

In [4]:
lei.calculate_LEI?

Signature:
lei.calculate_LEI(
    inputGHSL,
    old_list=[4, 5, 6],
    new_list=[3],
    buffer_dist=300,
)
Docstring:
Calculate LEI using vector objects in rasterio

INPUT
inputGHSL [string] - path the GHSL raster object
[optional] old_list [list of numbers] - values in GHSL to consider old urban. 4, 5, 6 indicates change from 2000
[optional] new_list [int] - value in GHSL to consider new urban. 3 indicates 2014

RETURNS
[array] - individual new built areas with LEI results. Each item is a single new built feature with three columns: 
            1. geometry of the new built area feature
            2. number of pixels in new built area donut from old built area
            3. area of new built area buffer
            
EXAMPLE
    # This calculates the change from 1990 and 2000
    lei_raw = calculate_LEI(input_ghsl, old_list = [5,6], new_list=[4])
    lei_90_00 = pd.DataFrame(lei_raw, columns=['geometry', 'old', 'total'])
    lei_90_00['LEI'] = lei_90_00['old'] / lei_90_00['total']

In [ ]:
# This calculates the change from 1990 and 2000
lei_raw = lei.calculate_LEI(input_ghsl, old_list=[5, 6], new_list=[4])
lei_90_00 = pd.DataFrame(lei_raw, columns=["geometry", "old", "total"])
lei_90_00["LEI"] = lei_90_00["old"] / lei_90_00["total"]
lei_90_00.head()

In [ ]:
# This calculates the change from 2000 and 2014
lei_raw = lei.calculate_LEI(input_ghsl, old_list=[4, 5, 6], new_list=[3])
lei_00_14 = pd.DataFrame(lei_raw, columns=["geometry", "old", "total"])
lei_00_14["LEI"] = lei_00_14["old"] / lei_00_14["total"]
lei_00_14.head()

In [ ]:
importlib.reload(lei)
# Calculate summaries of lei
lei.summarize_LEI(lei_90_00, leap_val=0.05, exp_val=0.75) / 1000000

In [ ]:
# Calculate summaries of lei
lei.summarize_LEI(lei_00_14, leap_val=0.05, exp_val=0.75) / 1000000

In [ ]:
# write raw LEI results to file
lei_90_00.to_csv(os.path.join(input_folder, "GHSL_LEI_90_00.csv"))
lei_00_14.to_csv(os.path.join(input_folder, "GHSL_LEI_00_14.csv"))

# Re-run analysis using the World Settlement Footprint

In [2]:
# Define input variables
input_folder = "/home/wb411133/temp/LOME"
if not os.path.exists(input_folder):
    os.makedirs(input_folder)
input_WSF = os.path.join(input_folder, "LOME_WSF.tif")
input_WSF_proj = os.path.join(input_folder, "LOME_WSF_PROJ.tif")

# This section will extract GHSL data from the global file, if you have the GHSL for the AOI extracted
#    define above as input_ghsl
if not os.path.exists(input_WSF_proj):
    # clip from global GHSL file
    wsf = "/home/public/Data/GLOBAL/WSF/Togo/Togo_WSF_evolution.tif"
    aoi = os.path.join(input_folder, "Grand_lome_dissolve.shp")
    in_ghsl = rasterio.open(wsf)
    inA = gpd.read_file(aoi)
    if not inA.crs == in_ghsl.crs:
        inA = inA.to_crs(in_ghsl.crs)
    rMisc.clipRaster(in_ghsl, inA, input_WSF)
    # WSF is stored in WGS84, making buffering and area calculations impossible.
    #    Instead we will standardize to the GHSL raster
    ghsl_raster = os.path.join(input_folder, "GHSL.tif")
    in_wsf = rasterio.open(input_WSF)
    in_ghsl = rasterio.open(ghsl_raster)
    rMisc.standardizeInputRasters(in_wsf, in_ghsl, input_WSF_proj, "C")

In [3]:
# This calculates the change from 1990 and 2000
lei_raw = lei.calculate_LEI(
    input_WSF_proj, old_list=list(range(1985, 1991)), new_list=list(range(1991, 2001))
)
lei_90_00 = pd.DataFrame(lei_raw, columns=["geometry", "old", "total"])
lei_90_00["LEI"] = lei_90_00["old"] / lei_90_00["total"]
lei_90_00.head()

,geometry,old,total,LEI
0,"POLYGON ((131490 702030, 131490 701970, 131520...",7,376,0.018617
1,"POLYGON ((131400 701970, 131400 701940, 131430...",7,356,0.019663
2,"POLYGON ((131460 701970, 131460 701940, 131490...",7,356,0.019663
3,"POLYGON ((131400 701880, 131400 701850, 131460...",7,376,0.018617
4,"POLYGON ((132510 701370, 132510 701340, 132540...",6,356,0.016854


In [4]:
# This calculates the change from 2000 and 2015
lei_raw = lei.calculate_LEI(
    input_WSF_proj, old_list=list(range(1985, 2001)), new_list=list(range(2001, 2016))
)
lei_00_14 = pd.DataFrame(lei_raw, columns=["geometry", "old", "total"])
lei_00_14["LEI"] = lei_00_14["old"] / lei_00_14["total"]
lei_00_14.head()

,geometry,old,total,LEI
0,"POLYGON ((131430 702030, 131430 701970, 131460...",13,376,0.034574
1,"POLYGON ((131370 701970, 131370 701850, 131400...",13,416,0.031250
2,"POLYGON ((131490 701970, 131490 701940, 131520...",13,356,0.036517
3,"POLYGON ((132480 701400, 132480 701370, 132540...",10,376,0.026596
4,"POLYGON ((132540 701370, 132540 701310, 132570...",10,457,0.021882


In [5]:
# Calculate summaries of lei
lei.summarize_LEI(lei_90_00, leap_val=0.001, exp_val=0.5) / 1000000

class
Expansion    6.6150
Infill       5.6961
Leapfrog     0.0108
Name: area, dtype: float64

In [6]:
# Calculate summaries of lei
lei.summarize_LEI(lei_00_14, leap_val=0.001, exp_val=0.5) / 1000000

class
Expansion    23.0148
Infill       19.2915
Leapfrog      0.1269
Name: area, dtype: float64

In [ ]:
# write raw LEI results to file
lei_90_00.to_csv(os.path.join(input_folder, "WSF_LEI_90_00.csv"))
lei_00_14.to_csv(os.path.join(input_folder, "WSF_LEI_00_14.csv"))

zip -r BGD_1KM.zip BGD_URBAN_DATA_new_naming/FINAL_STANDARD_1KM
zip -r BGD_STANDARD.zip BGD_URBAN_DATA_new_naming/FINAL_STANDARD
zip -r TZA_1KM.zip TZA_URBAN_DATA_new_naming/FINAL_STANDARD_1KM
zip -r TZA_STANDARD.zip TZA_URBAN_DATA_new_naming/FINAL_STANDARD
zip -r GHA_1KM.zip GHA_URBAN_DATA_new_naming/FINAL_STANDARD_1KM
zip -r GHA_STANDARD.zip GHA_URBAN_DATA_new_naming/FINAL_STANDARD
zip -r ETH_1KM.zip ETH_URBAN_DATA_new_naming/FINAL_STANDARD_1KM
zip -r ETH_STANDARD.zip ETH_URBAN_DATA_new_naming/FINAL_STANDARD
zip -r EGY_1KM.zip EGY_URBAN_DATA_new_naming/FINAL_STANDARD_1KM
zip -r EGY_STANDARD.zip EGY_URBAN_DATA_new_naming/FINAL_STANDARD
zip -r COL_1KM.zip COL_URBAN_DATA_new_naming/FINAL_STANDARD_1KM
zip -r COL_STANDARD.zip COL_URBAN_DATA_new_naming/FINAL_STANDARD
zip -r AGO_1KM.zip AGO_URBAN_DATA_new_naming/FINAL_STANDARD_1KM
zip -r AGO_STANDARD.zip AGO_URBAN_DATA_new_naming/FINAL_STANDARD
zip -r VNM_1KM.zip VNM_URBAN_DATA_new_naming/FINAL_STANDARD_1KM
zip -r VNM_STANDARD.zip VNM_URBAN

In [13]:
"%s_%s.zip" % (cur_dir.split("/")[-2].split("_")[0], cur_dir.split("_")[-1])

'VNM_STANDARD.zip'

In [12]:
cur_dir.split("_")[-1]

'STANDARD'